In [1]:
from collections import namedtuple
Line = namedtuple('Line', ['label', 'opcode', 'operand'])


In [2]:
def parse_line(line):
    parts = line.split()
    if len(parts) == 1:
        return Line(None, *parts, None)
    if len(parts) == 2:
        if parts[-1] == 'START':
            return Line(*parts, None)
        return Line(None, *parts)
    return Line(*parts)

def to_hex(addr, l = 4):
    return hex(addr)[2:].rjust(l, '0')


In [3]:
class Assembler:
    optab = {'STA':'23', 'LDA':'00', 'LDVH':'50', 'DIV':'24', 'ADD':'18', 'MUL':'20', 'SUB':'1C'}

    def __init__(self, program):
        self.symtab = dict()
        self.lines = list(map(parse_line, program.split('\n')))
        self.machine = ""
        self.convert()

    def convert(self) -> None:
        """Convert to object code and store in  self.machine"""
        pass


In [4]:
class SP_Assembler(Assembler):

    def convert(self):
        start = self.lines[0]
        self.name = start.label
        self.start = int(start.operand, 16) if start.operand else 0
        self.ptr = self.start

        for label, opcode, operand in self.lines[1:]:
            if opcode == 'END':
                self.size = self.ptr - self.start
            if opcode in self.optab:
                self.machine += 'T' + to_hex(self.ptr) + self.optab[opcode]
                if operand in self.symtab:
                    self.machine += to_hex(self.symtab[operand])
                else:
                    self.machine += to_hex(ord(operand))
                self.ptr += 3
                self.machine += '\n'
            else:
                if opcode =='RESW':
                    self.symtab[label] = self.ptr
                    self.ptr += int(operand) * 3
                elif opcode == 'RESB':
                    self.symtab[label] = self.ptr
                    self.ptr += int(operand)
                elif opcode == 'BYTE':
                    self.symtab[label] = self.ptr
                    self.ptr += len(bytes(operand, 'ascii'))
                elif opcode == 'WORD':
                    self.symtab[label] = self.ptr
                    self.ptr += 3
        self.machine.strip('\n')
        self.machine = "H"+self.name+to_hex(self.start)+ to_hex(self.size, 2) + '\n' + self.machine
        self.machine += 'E' + to_hex(self.start)


In [5]:
program = """COPY START 1000
ALPHA RESW 1
BETA RESW 1
LDA ALPHA
STA BETA
END"""


In [6]:
print(SP_Assembler(program).machine)


HCOPY10000c
T1006001000
T1009231003
E1000


In [7]:
class MP_Assembler(Assembler):
    def convert(self):
        self.pass_1()
        self.pass_2()

    def pass_1(self):
        start = self.lines[0]
        self.name = start.label
        self.start = int(start.operand, 16) if start.operand else 0
        self.ptr = self.start

        for label, opcode, operand in self.lines[1:]:
            if label:
                self.symtab[label] = self.ptr
            if opcode in self.optab:
                self.ptr += 3
            else:
                if opcode =='RESW':
                    self.symtab[label] = self.ptr
                    self.ptr += int(operand) * 3
                elif opcode == 'RESB':
                    self.symtab[label] = self.ptr
                    self.ptr += int(operand)
                elif opcode == 'BYTE':
                    self.symtab[label] = self.ptr
                    self.ptr += len(bytes(operand, 'ascii'))
                elif opcode == 'WORD':
                    self.symtab[label] = self.ptr
                    self.ptr += 3

    def pass_2(self):
        self.ptr = self.start
        for label, opcode, operand in self.lines[1:]:
            if opcode == 'END':
                self.size = self.ptr - self.start
            if opcode in self.optab:
                self.machine += 'T' + to_hex(self.ptr) + self.optab[opcode]
                if operand in self.symtab:
                    self.machine += to_hex(self.symtab[operand])
                else:
                    self.machine += to_hex(ord(operand))
                self.ptr += 3
                self.machine += '\n'
            else:
                if opcode =='RESW':
                    self.ptr += int(operand) * 3
                elif opcode == 'RESB':
                    self.ptr += int(operand)
                elif opcode == 'BYTE':
                    self.ptr += len(bytes(operand, 'ascii'))
                elif opcode == 'WORD':
                    self.ptr += 3
        self.machine.strip('\n')
        self.machine = "H"+self.name+to_hex(self.start)+ to_hex(self.size, 2) + '\n' + self.machine
        self.machine += 'E' + to_hex(self.start)


In [8]:
program = """COPY START 1000
LDA ALPHA
STA BETA
ALPHA RESW 1
BETA RESW 1
END"""


In [9]:
print(MP_Assembler(program).machine)


HCOPY10000c
T1000001006
T1003231009
E1000
